## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Mataura,NZ,-46.1927,168.8643,38.48,95,100,0.38,overcast clouds
1,Richards Bay,ZA,-28.7830,32.0377,66.22,86,67,7.14,broken clouds
2,Flinders,AU,-34.5833,150.8552,55.38,88,95,5.99,light rain
3,Puerto Ayora,EC,-0.7393,-90.3518,71.55,87,58,5.99,broken clouds
4,Atuona,PF,-9.8000,-139.0333,77.56,72,15,15.64,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,Atuona,PF,-9.8000,-139.0333,77.56,72,15,15.64,few clouds
18,Mashhad,IR,36.2970,59.6062,78.94,31,0,0.00,clear sky
19,Hobyo,SO,5.3505,48.5268,75.92,83,26,25.52,scattered clouds
23,Bandarbeyla,SO,9.4942,50.8122,75.67,84,24,28.03,few clouds
27,Eureka,US,40.8021,-124.1637,83.68,76,68,8.01,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City                   247
Country                247
Lat                    247
Lng                    247
Max Temp               247
Humidity               247
Cloudiness             247
Wind Speed             247
Current Description    247
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,77.56,few clouds,-9.8000,-139.0333,
18,Mashhad,IR,78.94,clear sky,36.2970,59.6062,
19,Hobyo,SO,75.92,scattered clouds,5.3505,48.5268,
23,Bandarbeyla,SO,75.67,few clouds,9.4942,50.8122,
27,Eureka,US,83.68,broken clouds,40.8021,-124.1637,
31,Acapulco,MX,85.87,overcast clouds,16.8634,-99.8901,
36,Butaritari,KI,80.82,light rain,3.0707,172.7902,
45,Trairi,BR,76.62,broken clouds,-3.2778,-39.2689,
46,Jamestown,US,85.05,clear sky,42.0970,-79.2353,
48,Hilo,US,82.27,broken clouds,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add latitude and longitude into our parameters
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Places API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df.loc[hotel_df["Hotel Name"].eq("")].index)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,77.56,few clouds,-9.8000,-139.0333,Villa Enata
18,Mashhad,IR,78.94,clear sky,36.2970,59.6062,Khanye Sabz Hotel
19,Hobyo,SO,75.92,scattered clouds,5.3505,48.5268,Iftin hotel hobyo
23,Bandarbeyla,SO,75.67,few clouds,9.4942,50.8122,JABIR HOTEL
27,Eureka,US,83.68,broken clouds,40.8021,-124.1637,Eureka Inn Trademark By Wyndham
31,Acapulco,MX,85.87,overcast clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
36,Butaritari,KI,80.82,light rain,3.0707,172.7902,Isles Sunset Lodge
45,Trairi,BR,76.62,broken clouds,-3.2778,-39.2689,janelas do mar
46,Jamestown,US,85.05,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
48,Hilo,US,82.27,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))